# Fake News Detection - Phase 4: Deep Learning Models

---

### Objectifs de ce notebook

Ce notebook implemente les modeles de deep learning pour la detection de fake news, conformement a la methodologie de l'article de reference (Roumeliotis et al., 2025). Ces modeles seront compares aux baselines classiques etablies precedemment.

Modeles implementes:
1. CNN (Convolutional Neural Network) pour le texte
2. BERT (Bidirectional Encoder Representations from Transformers)

Selon l'article de reference, le CNN a atteint 58.6% d'accuracy tandis que les modeles bases sur les transformers (BERT, GPT) atteignent des performances superieures.

## 1. Configuration de l'environnement

In [ ]:
# Installation des dependances
# !pip install tensorflow transformers torch scikit-learn pandas numpy matplotlib

In [2]:
# Importation des bibliotheques
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import pickle
import warnings

# Deep Learning - TensorFlow/Keras pour CNN
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import (
    Embedding, Conv1D, GlobalMaxPooling1D, Dense, Dropout
)
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.optimizers import Adam

# Metriques
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    confusion_matrix, classification_report
)

warnings.filterwarnings('ignore')
plt.style.use('seaborn-v0_8-whitegrid')

print(f"TensorFlow version: {tf.__version__}")
print(f"GPU disponible: {tf.config.list_physical_devices('GPU')}")

TensorFlow version: 2.20.0
GPU disponible: []


In [4]:
pip install tensorflow


  Using cached tensorflow-2.20.0-cp311-cp311-win_amd64.whl.metadata (4.6 kB)
Using cached tensorflow-2.20.0-cp311-cp311-win_amd64.whl (331.8 MB)


## 2. Chargement des donnees

In [3]:
# Chargement des datasets preprocesses
DATA_PATH = "../data/interim/"

train_df = pd.read_csv(DATA_PATH + "train.csv")
val_df = pd.read_csv(DATA_PATH + "validation.csv")
test_df = pd.read_csv(DATA_PATH + "test.csv")

print(f"Training set:   {len(train_df):,} samples")
print(f"Validation set: {len(val_df):,} samples")
print(f"Test set:       {len(test_df):,} samples")

Training set:   24,732 samples
Validation set: 6,183 samples
Test set:       7,729 samples


In [4]:
# Preparation des donnees
X_train = train_df['text'].values
y_train = train_df['label'].values

X_val = val_df['text'].values
y_val = val_df['label'].values

X_test = test_df['text'].values
y_test = test_df['label'].values

print(f"Donnees preparees.")

Donnees preparees.


## 3. CNN (Convolutional Neural Network)

### 3.1 Theorie

Les CNN appliques au texte utilisent des filtres de convolution pour capturer les patterns locaux (n-grams) dans les sequences de mots. L'architecture typique comprend:

1. **Embedding Layer**: Transforme les indices de mots en vecteurs denses
2. **Conv1D Layer**: Applique des filtres pour detecter des patterns
3. **GlobalMaxPooling**: Extrait les features les plus saillantes
4. **Dense Layers**: Classification finale

Reference: Kaliyar et al. (2020) - FNDNet

### 3.2 Tokenisation et preparation des sequences

In [5]:
# Parametres de tokenisation
MAX_VOCAB_SIZE = 10000    # Taille maximale du vocabulaire
MAX_SEQUENCE_LENGTH = 512  # Longueur maximale des sequences (en tokens)
EMBEDDING_DIM = 128        # Dimension des embeddings

# Initialisation du tokenizer Keras
tokenizer = Tokenizer(
    num_words=MAX_VOCAB_SIZE,
    oov_token='<OOV>'  # Token pour les mots hors vocabulaire
)

# Entrainement du tokenizer sur les donnees d'entrainement
tokenizer.fit_on_texts(X_train)

print(f"Taille du vocabulaire: {len(tokenizer.word_index):,} mots")
print(f"Vocabulaire limite a: {MAX_VOCAB_SIZE:,} mots")

Taille du vocabulaire: 116,489 mots
Vocabulaire limite a: 10,000 mots


In [6]:
# Conversion des textes en sequences d'indices
X_train_seq = tokenizer.texts_to_sequences(X_train)
X_val_seq = tokenizer.texts_to_sequences(X_val)
X_test_seq = tokenizer.texts_to_sequences(X_test)

# Padding des sequences pour uniformiser la longueur
X_train_pad = pad_sequences(X_train_seq, maxlen=MAX_SEQUENCE_LENGTH, padding='post')
X_val_pad = pad_sequences(X_val_seq, maxlen=MAX_SEQUENCE_LENGTH, padding='post')
X_test_pad = pad_sequences(X_test_seq, maxlen=MAX_SEQUENCE_LENGTH, padding='post')

print(f"Shape des sequences:")
print(f"  Training:   {X_train_pad.shape}")
print(f"  Validation: {X_val_pad.shape}")
print(f"  Test:       {X_test_pad.shape}")

Shape des sequences:
  Training:   (24732, 512)
  Validation: (6183, 512)
  Test:       (7729, 512)


### 3.3 Architecture du modele CNN

In [ ]:
def create_cnn_model(vocab_size, embedding_dim, max_length):
    """
    Cree un modele CNN pour la classification de texte.
    
    Architecture basee sur l'article de reference:
    - Embedding -> Conv1D -> GlobalMaxPooling -> Dense -> Output
    """
    model = Sequential([
        # Couche d'embedding
        Embedding(
            input_dim=vocab_size,
            output_dim=embedding_dim,
            input_length=max_length
        ),
        
        # Couche de convolution 1D
        Conv1D(
            filters=128,
            kernel_size=5,
            activation='relu'
        ),
        
        # Global Max Pooling
        GlobalMaxPooling1D(),
        
        # Couches denses
        Dense(128, activation='relu'),
        Dropout(0.5),
        Dense(64, activation='relu'),
        Dropout(0.3),
        
        # Couche de sortie
        Dense(1, activation='sigmoid')
    ])
    
    return model

print("Fonction de creation du modele CNN definie.")

In [ ]:
# Creation du modele
cnn_model = create_cnn_model(
    vocab_size=MAX_VOCAB_SIZE,
    embedding_dim=EMBEDDING_DIM,
    max_length=MAX_SEQUENCE_LENGTH
)

# Compilation du modele
# Hyperparametres conformes a l'article: learning_rate=2e-5
cnn_model.compile(
    optimizer=Adam(learning_rate=2e-5),
    loss='binary_crossentropy',
    metrics=['accuracy']
)

# Resume de l'architecture
print("Architecture du modele CNN:")
cnn_model.summary()

### 3.4 Entrainement du CNN

In [ ]:
# Configuration de l'entrainement
# Conformement a l'article: 3 epochs, batch_size=6
EPOCHS = 3
BATCH_SIZE = 6

# Callbacks pour l'entrainement
callbacks = [
    EarlyStopping(
        monitor='val_loss',
        patience=2,
        restore_best_weights=True
    )
]

print(f"Configuration:")
print(f"  Epochs: {EPOCHS}")
print(f"  Batch size: {BATCH_SIZE}")
print(f"  Learning rate: 2e-5")

In [ ]:
# Entrainement du modele
print("Entrainement du CNN en cours...")

cnn_history = cnn_model.fit(
    X_train_pad, y_train,
    validation_data=(X_val_pad, y_val),
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    callbacks=callbacks,
    verbose=1
)

print("\nEntrainement termine.")

### 3.5 Visualisation de l'entrainement

In [ ]:
# Visualisation des courbes d'apprentissage
fig, axes = plt.subplots(1, 2, figsize=(12, 4))

# Loss
axes[0].plot(cnn_history.history['loss'], label='Training Loss')
axes[0].plot(cnn_history.history['val_loss'], label='Validation Loss')
axes[0].set_title('CNN - Evolution de la Loss')
axes[0].set_xlabel('Epoch')
axes[0].set_ylabel('Loss')
axes[0].legend()

# Accuracy
axes[1].plot(cnn_history.history['accuracy'], label='Training Accuracy')
axes[1].plot(cnn_history.history['val_accuracy'], label='Validation Accuracy')
axes[1].set_title('CNN - Evolution de l\'Accuracy')
axes[1].set_xlabel('Epoch')
axes[1].set_ylabel('Accuracy')
axes[1].legend()

plt.tight_layout()
plt.show()

### 3.6 Evaluation du CNN

In [ ]:
# Predictions sur le test set
cnn_pred_proba = cnn_model.predict(X_test_pad, verbose=0)
cnn_pred = (cnn_pred_proba > 0.5).astype(int).flatten()

# Calcul des metriques
cnn_metrics = {
    'model': 'CNN',
    'accuracy': accuracy_score(y_test, cnn_pred),
    'precision': precision_score(y_test, cnn_pred),
    'recall': recall_score(y_test, cnn_pred),
    'f1_score': f1_score(y_test, cnn_pred)
}

print("CNN - Resultats sur le TEST SET:")
print("="*50)
for metric, value in cnn_metrics.items():
    if metric != 'model':
        print(f"  {metric}: {value:.4f}")

In [ ]:
# Matrice de confusion
cm = confusion_matrix(y_test, cnn_pred)

plt.figure(figsize=(6, 5))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=['Fake (0)', 'True (1)'],
            yticklabels=['Fake (0)', 'True (1)'])
plt.title('Matrice de confusion - CNN')
plt.ylabel('Valeur reelle')
plt.xlabel('Prediction')
plt.tight_layout()
plt.show()

## 4. BERT (Bidirectional Encoder Representations from Transformers)

### 4.1 Theorie

BERT est un modele de langage pre-entraine base sur l'architecture Transformer. Ses avantages incluent:

1. **Comprehension bidirectionnelle**: Considere le contexte des deux cotes d'un mot
2. **Pre-entrainement massif**: Entraine sur de grands corpus (Wikipedia, BookCorpus)
3. **Transfer learning**: Fine-tuning efficace pour des taches specifiques

L'article de reference rapporte 97.5% d'accuracy avec BERT fine-tune.

In [ ]:
# Importation des bibliotheques pour BERT
try:
    from transformers import (
        BertTokenizer, BertForSequenceClassification,
        Trainer, TrainingArguments
    )
    import torch
    from torch.utils.data import Dataset, DataLoader
    
    BERT_AVAILABLE = True
    print(f"PyTorch version: {torch.__version__}")
    print(f"CUDA disponible: {torch.cuda.is_available()}")
except ImportError:
    BERT_AVAILABLE = False
    print("Transformers ou PyTorch non disponible.")
    print("Installez avec: pip install transformers torch")

### 4.2 Preparation des donnees pour BERT

In [ ]:
# Cette section ne s'execute que si BERT est disponible

# Chargement du tokenizer BERT
bert_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Parametres
BERT_MAX_LENGTH = 512  # Limite de BERT

print(f"Tokenizer BERT charge.")
print(f"Longueur maximale: {BERT_MAX_LENGTH}")

In [ ]:
# Classe Dataset personnalisee pour PyTorch
class FakeNewsDataset(Dataset):
    """
    Dataset PyTorch pour la classification de fake news avec BERT.
    """
    def __init__(self, texts, labels, tokenizer, max_length):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length
    
    def __len__(self):
        return len(self.texts)
    
    def __getitem__(self, idx):
        text = str(self.texts[idx])
        label = self.labels[idx]
        
        encoding = self.tokenizer(
            text,
            truncation=True,
            padding='max_length',
            max_length=self.max_length,
            return_tensors='pt'
        )
        
        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(label, dtype=torch.long)
        }

print("Classe FakeNewsDataset definie.")

In [ ]:
# Creation des datasets
train_dataset = FakeNewsDataset(X_train, y_train, bert_tokenizer, BERT_MAX_LENGTH)
val_dataset = FakeNewsDataset(X_val, y_val, bert_tokenizer, BERT_MAX_LENGTH)
test_dataset = FakeNewsDataset(X_test, y_test, bert_tokenizer, BERT_MAX_LENGTH)

print(f"Datasets crees:")
print(f"  Training:   {len(train_dataset)} samples")
print(f"  Validation: {len(val_dataset)} samples")
print(f"  Test:       {len(test_dataset)} samples")

### 4.3 Chargement et configuration du modele BERT

In [ ]:
# Chargement du modele BERT pre-entraine
bert_model = BertForSequenceClassification.from_pretrained(
    'bert-base-uncased',
    num_labels=2  # Classification binaire
)

print(f"Modele BERT charge.")
print(f"Nombre de parametres: {sum(p.numel() for p in bert_model.parameters()):,}")

### 4.4 Configuration de l'entrainement

In [ ]:
# Arguments d'entrainement conformes a l'article
training_args = TrainingArguments(
    output_dir='./models/bert',
    num_train_epochs=3,              # 3 epochs comme dans l'article
    per_device_train_batch_size=6,   # Batch size = 6
    per_device_eval_batch_size=6,
    learning_rate=2e-5,              # Learning rate = 2e-5
    warmup_steps=100,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=50,
    eval_strategy='epoch',
    save_strategy='epoch',
    load_best_model_at_end=True,
    metric_for_best_model='f1'
)

print("Arguments d'entrainement configures.")

In [ ]:
# Fonction pour calculer les metriques
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    
    return {
        'accuracy': accuracy_score(labels, predictions),
        'precision': precision_score(labels, predictions),
        'recall': recall_score(labels, predictions),
        'f1': f1_score(labels, predictions)
    }

print("Fonction compute_metrics definie.")

In [ ]:
# Creation du Trainer
trainer = Trainer(
    model=bert_model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics
)

print("Trainer cree.")

### 4.5 Entrainement du modele BERT

In [ ]:
# Entrainement
print("Entrainement de BERT en cours...")
print("(Cette operation peut prendre plusieurs minutes sur GPU)")

train_result = trainer.train()

print("\nEntrainement termine.")
print(f"Training loss finale: {train_result.training_loss:.4f}")

### 4.6 Evaluation du modele BERT

In [ ]:
# Evaluation sur le test set
print("Evaluation de BERT sur le test set...")

test_results = trainer.evaluate(test_dataset)

bert_metrics = {
    'model': 'BERT',
    'accuracy': test_results['eval_accuracy'],
    'precision': test_results['eval_precision'],
    'recall': test_results['eval_recall'],
    'f1_score': test_results['eval_f1']
}

print("\nBERT - Resultats sur le TEST SET:")
print("="*50)
for metric, value in bert_metrics.items():
    if metric != 'model':
        print(f"  {metric}: {value:.4f}")

In [ ]:
# Predictions pour la matrice de confusion
bert_predictions = trainer.predict(test_dataset)
bert_pred = np.argmax(bert_predictions.predictions, axis=1)

# Matrice de confusion
cm_bert = confusion_matrix(y_test, bert_pred)

plt.figure(figsize=(6, 5))
sns.heatmap(cm_bert, annot=True, fmt='d', cmap='Greens',
            xticklabels=['Fake (0)', 'True (1)'],
            yticklabels=['Fake (0)', 'True (1)'])
plt.title('Matrice de confusion - BERT')
plt.ylabel('Valeur reelle')
plt.xlabel('Prediction')
plt.tight_layout()
plt.show()

## 5. Comparaison des modeles Deep Learning

In [ ]:
# Compilation des resultats
deep_results = pd.DataFrame([cnn_metrics, bert_metrics])
deep_results = deep_results.set_index('model')

print("Comparaison des modeles deep learning:")
print("="*60)
print(deep_results.round(4))

In [ ]:
# Visualisation comparative
fig, ax = plt.subplots(figsize=(10, 6))

metrics_to_plot = ['accuracy', 'precision', 'recall', 'f1_score']
x = np.arange(len(metrics_to_plot))
width = 0.35

# CNN
cnn_values = deep_results.loc['CNN', metrics_to_plot].values
ax.bar(x - width/2, cnn_values, width, label='CNN', color='#3498db')

# BERT
bert_values = deep_results.loc['BERT', metrics_to_plot].values
ax.bar(x + width/2, bert_values, width, label='BERT', color='#27ae60')

ax.set_ylabel('Score')
ax.set_title('Comparaison CNN vs BERT')
ax.set_xticks(x)
ax.set_xticklabels(['Accuracy', 'Precision', 'Recall', 'F1-Score'])
ax.legend()
ax.set_ylim(0, 1.05)

plt.tight_layout()
plt.show()

### Interpretation des resultats

Conformement aux observations de l'article de reference (Roumeliotis et al., 2025):

1. Le CNN presente des performances limitees pour la classification de fake news, principalement en raison de sa difficulte a capturer les relations semantiques complexes

2. BERT surpasse significativement le CNN grace a sa comprehension contextuelle bidirectionnelle et son pre-entrainement massif

3. La difference de performance souligne l'importance des modeles bases sur les transformers pour les taches de NLP complexes comme la detection de fake news

## 6. Sauvegarde des modeles

In [ ]:
# Creation des repertoires
MODEL_PATH = "./models/deep_learning/"
os.makedirs(MODEL_PATH, exist_ok=True)

# Sauvegarde du modele CNN
cnn_model.save(MODEL_PATH + "cnn_model.h5")

# Sauvegarde du tokenizer Keras
with open(MODEL_PATH + "keras_tokenizer.pkl", 'wb') as f:
    pickle.dump(tokenizer, f)

print(f"Modele CNN sauvegarde: {MODEL_PATH}cnn_model.h5")
print(f"Tokenizer sauvegarde: {MODEL_PATH}keras_tokenizer.pkl")

In [ ]:
# Sauvegarde du modele BERT
bert_model.save_pretrained(MODEL_PATH + "bert_model/")
bert_tokenizer.save_pretrained(MODEL_PATH + "bert_tokenizer/")

print(f"Modele BERT sauvegarde: {MODEL_PATH}bert_model/")
print(f"Tokenizer BERT sauvegarde: {MODEL_PATH}bert_tokenizer/")

In [ ]:
# Sauvegarde des resultats
deep_results.to_csv(MODEL_PATH + "deep_learning_results.csv")
print(f"\nResultats sauvegardes: {MODEL_PATH}deep_learning_results.csv")

## 7. Resume

In [ ]:
# Generation du resume
print("="*60)
print("RESUME - MODELES DEEP LEARNING")
print("="*60)

print("\n1. CNN (Convolutional Neural Network)")
print(f"   - Architecture: Embedding + Conv1D + Dense")
print(f"   - Parametres: ~{cnn_model.count_params():,}")
print(f"   - F1-Score: {cnn_metrics['f1_score']:.4f}")

print("\n2. BERT (bert-base-uncased)")
print(f"   - Architecture: Transformer encoder")
print(f"   - Parametres: ~110M")
print(f"   - F1-Score: {bert_metrics['f1_score']:.4f}")

print("\n3. COMPARAISON")
print(deep_results.round(4).to_string())

print("\n4. CONCLUSION")
print("   BERT surpasse significativement le CNN pour la detection de fake news,")
print("   confirmant les observations de l'article de reference.")

---

## Prochaine etape

Le notebook suivant (`fnd_05_cross_validation.ipynb`) implementera la validation croisee pour une evaluation plus robuste des modeles.

---

**References:**
- Roumeliotis, K.I., Tselikas, N.D., & Nasiopoulos, D.K. (2025). Fake News Detection and Classification. *Future Internet*, 17, 28.
- Kaliyar, R.K. et al. (2020). FNDNet - A Deep CNN for Fake News Detection. *Cognitive Systems Research*.
- Devlin, J. et al. (2019). BERT: Pre-training of Deep Bidirectional Transformers. *NAACL-HLT*.